In [ ]:
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from cleans import clean_meters_property_name, clean_energy_property_name, clean_area_property_name, \
    clean_money_property_name, clean_weight_property_name

In [ ]:
database = "db_ua"
user = "apiv2"
password = "35GMHdDP9DLuKtnV"
host = "34.141.85.48"

In [ ]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

In [ ]:
df = pd.read_sql_table("aimap_object_properties", engine)


In [ ]:
def clean_units(df):
    # df["units"] = df["units"].str.replace("(\d*,\d*\s)|(|,)(\d*\s)|(|,)(\d*\W\d*)|(|,)(\d*\w)|(|,)(\d*,\d*)", "", flags=re.IGNORECASE)
    df["units"] = df["units"].str.replace(
        "((^[\d.,\s\/-]*)|(\d*\(.*-\s\d*,\d*\s)|(^Шпаклювання)\s*\W*\d*,\d*\s|()(^гараж\d*,\d*\s)|"
        "(^Шпаклівка\s*\W*\s\d*,\s\d*\s*)|(^Складські)\s\W*\d*|\W*,\s|\W*\s\d*,\d*\s|^виробництво\sплит\sДСП|"
        "^виробництво\s\W*\w*|\(\W*\)\s|\sзберігання|\s-\W\s\W*)", "", flags=re.IGNORECASE)
    df["units"] = df["units"].str.replace(
        "(\s\(основна\)|\(підсобна\)|\sкотельні|сушильних\sкамер|"
        "\(І\sчерга.*|приміщення\s\d*|\(складських.*|"
        "\(.*|^Офісно-\W+\d\W\s\W+\d*,\W\d;\s\W*\d\W\s\W*\d*\s)", "", flags=re.IGNORECASE)
    df["units"] = df["units"].str.replace(
        "((|,)(поверх\s\d*-го\sблоку\s\d\Wти\sповерхової\sбудівлі)$|"
        "(|,)(на четвертому поверсі чотириповерхового житлового будинку)$|(|,)(х\d*)$|(|,)(\Wтехнічний поверх)$|"
        "(|,)(будівлі\sз\sтехнічним\sповерхом\s\d*,\d*)$|(|,)(цокольмансарда)$|"
        "(|,)(на\sпершому\sповерсі\sчотириповерхового\sжитлового\sбудинку)$|(|,)(з\sмансардою)$|(|,)(\W\s\d*,\d*)$|"
        "(|,)()$|(|,)((\Wмансарда\Wпідвал|\Wмансарда))$|(|,)(третій\sповерх\sчотирьохповерхової\sбудівлі)$|"
        "(|,)(й\sповерх\sта\sдругий\sрівень\s\d-го\sповерху)$|(|,)(%)$|(|,)(На\s\d\sповерсі \d-и\sповерхового\sбудинку)$|"
        "(|,)(цокольний\W\d*)$|(|,)(\)\s\d*\W\s\d\)\s\d*\W\s\d\)\s\d*)$|(|,)(\d*)$|(|,)(поверх\s\d\sповерхового\sбудинку)$|"
        "(|,)(т\sв\sзміну,теплова\sпотужність\sМвт0,6котел\sпаровий\sавтоматизований\sна\sтвердому\sпаливі\sз\sвбудованим"
        "\sекономайзером\s1штвид\sпалива-\sтверде\sпаливо)$|(|,)((один\sта\sмансарда|один\sз\sмансардою|з\sмансардою\sу"
        "\sдвох\sрівня|\W\sмансарда|\Wцокольний\Wмансардний|\Wмансардний\sповерх\sта\sпідвал|й\sповерх\sта\sвлаштований"
        "\sв\sрівні\s\d-го\s\d-й\sповер|з\sпідвалом|\Wпідвал|\Wтехповер|на\sпершому\sповерсі|підвальне\sв\sчотириповерхому"
        "\sжитловому\sбудинку|\W+1))$|(|,)(\Wпідвал)$|(|,)(Залізобетонні\sс\/п\sта\sдерев'яні\sшпали)$|(|,)(цоколь\W\d)$|"
        "(|,)(\W\d\W)$|(|,)(й\sповерх\s\d-ти\sповерхового\sжитлового\sбудинку)$|(повер)$|(з\sцокольним\sта\sмансардними\sповерхами)$|"
        "(цокольний\s\W\s\d\sповер\\|цокольний\s\W\s\d\sповер)$|(х\d*\sголів)$|(\Wмансарда\Wпогреб)$|(технічний\sповер)$|"
        "(\Wтехнічний)$|(\WМАНСАРДНИЙ)$|(|,)(поверхів\sз\sцокольним\sповерхом)$|(на першому  поверсі дев'ятиповерхового житлового будинку)$|"
        "(\W\s\d,\d)$|(х\d*)$|(цокольний поверх 5-ти поверхової будівлі)$|(металева конструкція)$|(індивідуальне газове)$|"
        "(металеві конструкції з утепленням)$|(плоска утеплена покрівляПВХ мембрана)$|(й\sта\sвлаштований\sу\sпросторі\s"
        "\d-го\s\sчетвертий\sповерхи)$|(на\sпершому\sповерсі\sдев'ятиповерхового\sжитлового\sбудинку)$|()$|(\+)$|"
        "((\+мансарний\+цокольний\+підвальний|\+мансардний\+підвальний\+цокольний))$|(на першому\sповерсі\s"
        "\d-поверхового\sжитлового\sбудинку)$|(й\s;\s\d-й)$|(\;)$|(й\sта\s\d-й)$|(камінь\sчерепашник)$|"
        "(одноповерхова квартира на 1-му поверсі 9-ти поверхового будинку)$|(Бетонні блоки)$|(й поверх \d-и поверхового будинку)$|"
        "(з підвалом і надбудовою)$|^(ІІ - основна будівляприбудова завантажувальної зони - ІІІ)$|(на \d-му\sповерсі\s"
        "\d-поверхового\sжитлового\sбудинку)$|(\W\sмансардний)$|(Цокольний\sповерх\sдвоповерхового\sбудинку)$|"
        "(з\sмансардним\sповерхом)$|(й\sповерх\s\d-поверхового\sбудинку)$|(на\sпершому\sповерсі\sодноповерхового\sжитлового\sбудинку)$|"
        "(;\s\d*\/\d*;\s\d\/\d;\s\d*;)$|((підвал\s\W|підвал\W|ПІДВАЛ\W))$|(з\sцоколем)$|(й та влаштований у просторі 4-го 5-й поверхи)$|"
        "(двоповерхова \W одноповерхова прибудова)$|(квартира розташована на третьому поверсі шестиповерхового багатоквартирного житлового будинку)$|"
        "(три з технічним поверхом з терасою та підвалом)$|(підвалу \d-ти пов. будівлі)$|(поверх житлового будинку)$|(поверхи з цокольним поверхом)$|"
        "(\Wцоколь)$|(з підвалом та мансардою)$|(й поверх \d-ти поверхового будинку)$|(\W\d*)$|(\W\s\d*,\d*\W)$|(\Wмансарда\Wцокольний)$|(\W\s\d*\W)$|"
        "(й та \d-й поверх \d-ти поверхового будинку)$|(цоколь\W)$|(на \d поверсі \d-поверхового житлового будинку)$|(підвал\W\d-ий)$|(\Wпідвальний)$|"
        "(з\sмансардним)$|(відвантаження в зміну)$|)", "", flags=re.IGNORECASE)
    # df["units"] = df["units"].str.replace("х\d*\sмм\d\s-\s\d,\d*", "мм2", flags=re.IGNORECASE)
    # df['units'] = df['units'].replace("м2)", "м2")
    # df['units'] = df['units'].replace("Шпаклювання стін та укосів, 356,05 м2", "м2")
    # df['units'] = df['units'].replace("Шпаклювання стін та укосів356,05 м2", "м2")
    # df['units'] = df['units'].replace("АВБбШВ 4х95 мм2", "м2")
    # df['units'] = df['units'].replace("АВБбШВ 4х35 мм2 - 29,0", "м2")
    # df['units'] = df['units'].replace("Шпаклівка стін0 м2", "м2")
    # df['units'] = df['units'].replace("гаражм2", "м2")
    # df['units'] = df['units'].replace("торгова площа 37,91м2", "м2")
    # df['units'] = df['units'].replace("торгівельна площам2", "м2")
    # df['units'] = df['units'].replace("Адмінбудинокм2", "м2")
    # df['units'] = df['units'].replace("Цех- складм2", "м2")
    # df['units'] = df['units'].replace("Допоміжна площам2", "м2")
    # df['units'] = df['units'].replace("Загальна площа", "м2")
    # df['units'] = df['units'].replace("(303,53м2 -1-й пусковий комплекс; 1144,80м2- 2-й пусковий комплекс)", "м2")
    # df['units'] = df['units'].replace("Заповнення віконних прорізів готовими блоками площею до 3 м2 з металопластику", "м2")
    return df

In [ ]:
property_df = df["property"]
# property_value_df = df["property_value"]
# full_df = property_value_df

In [ ]:
property_value_df = df[df["property_value"].notna()]

In [ ]:
property_value_df['property'].unique()

In [ ]:
property_value_df["normalize_property"] = property_value_df["property"]

In [ ]:
property_value_df.dropna(subset=['property_value'])

In [ ]:
property_value_df.value_counts()

In [ ]:
property_value_df['property_value'] = property_value_df['property_value'].str.strip()
property_value_df = property_value_df[property_value_df['property_value'] != ""]
property_value_df = property_value_df[property_value_df['property_value'] != " "]
property_value_df = property_value_df[property_value_df['property_value'] != "0"]
property_value_df = property_value_df[property_value_df['property_value'] != "-"]
property_value_df = property_value_df[property_value_df['property_value'] != " - "]
property_value_df = property_value_df[property_value_df['property_value'] != " -"]
property_value_df = property_value_df[property_value_df['property_value'] != ","]
property_value_df = property_value_df[property_value_df['property_value'] != "--"]
property_value_df = property_value_df[property_value_df['property_value'] != "0,0"]
property_value_df = property_value_df[property_value_df['property_value'] != "0,00"]
property_value_df = property_value_df[property_value_df['property_value'] != "0 чол."]
property_value_df = property_value_df[property_value_df['property_value'] != "0 куб.м."]
property_value_df = property_value_df[property_value_df['property_value'] != "0 чол."]
property_value_df = property_value_df[property_value_df['property_value'] != "0 Гкал"]
property_value_df = property_value_df[property_value_df['property_value'] != "0 кВт"]
property_value_df = property_value_df[property_value_df['property_value'] != "0 кв. м"]


In [ ]:
property_value_df.value_counts()

In [ ]:
#  # property_value_df.property == "Площа"
# a = property_value_df.loc[property_value_df['property'] == "Площа"]
# property_value_df["normalize_property"] = property_value_df["normalize_property"].str.replace("(\sсм$|\d\s\см|\sм\d|м.\d|мм\d|км|км.кв|м²|м³|кг|м.кв|кВт|тис.грн|грн|грн.|КВт|КВ.М.|(кв.м.)|\sкв.|м.кв|куб.м|\sод|МВт|мвт|\sшт|\sга|\sртир|(\sкуб.\sм.)|(\sкуб.)|\sтис.|\sга|(,\sм.)|(,\sтон)|(\sтон)|(,\sтон)|(\sтон)|(,\sкв.\sм.)|(,\sосіб\/день)|рік|\/|)", "")
# property_value_df['property'].unique()
# # property_value_df['normalize_property'].unique()

In [ ]:
property_value_df['normalize_property'].unique()

In [ ]:
clean_meters_property_name(property_value_df)
clean_energy_property_name(property_value_df)
clean_area_property_name(property_value_df)
clean_money_property_name(property_value_df)
clean_weight_property_name(property_value_df)

In [ ]:
property_value_df['normalize_property'].unique()
# Розливання бітумної емульсії (0,4 л/м2) т.
# УЛАШТУВАННЯ ОДНОШАРОВИХ АСФАЛЬТОБЕТОННИХ ПОКРИТТІВ ДОРІЖОК ТА ТРОТУАРІВ ІЗ ДРІБНОЗЕРНИСТОЇ АСФАЛЬТОБЕТОННОЇ СУМІШІ, КВ.М.

In [ ]:
import difflib
property_value_df['normalize_property'] = property_value_df['normalize_property'].str.strip()
property_value_df['normalize_property'] = property_value_df['normalize_property'].str.capitalize()

In [ ]:
property_value_df = property_value_df[property_value_df['normalize_property'] != ""]

In [ ]:
property_value_df['normalize_property'].unique()

In [ ]:
#  ---01.03---
# Розрахунковий електричний струм, а
# Довжина проїзної частини, м.
# Ухил ділянки, %
# Кількість робочих місць, роб.місце

In [ ]:
# a = property_value_df.iloc[[2]]["normalize_property"]
# b = property_value_df.iloc[[2]]["normalize_property"]
# for i,s in enumerate(difflib.ndiff(a, b)):
#     if s[0]==' ': continue
#     elif s[0]=='-':
#         print(u'Delete "{}" from position {}'.format(s[-1],i))
#     elif s[0]=='+':
#         print(u'Add "{}" to position {}'.format(s[-1],i))

In [28]:
property_value_df["units"] = property_value_df["property_value"]

C:\Users\dev-1\AppData\Local\Temp\ipykernel_6540\1367561054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_value_df["units"] = property_value_df["property_value"]


In [ ]:
clean_units(property_value_df)

In [29]:
property_value_df["property_value"].unique()

array(['26', '149296.6', '1', ..., '200.92', '60.49', '3141.72'],
      dtype=object)

In [ ]:
property_value_df["units"].unique()

In [ ]:

# +
# +мансарний+цокольний+підвальний
# +мансардний+підвальний+цокольний
# на першому поверсі 5-поверхового житлового будинку
# й ; 2-й
# ;
# й та 2-й
# камінь черепашник
# одноповерхова квартира на 1-му поверсі 9-ти поверхового будинку
# Бетонні блоки
# й поверх 3-и поверхового будинку
# з підвалом і надбудовою


In [ ]:
# property_value_df['units'].fillna(property_value_df['property'], inplace=True)

In [ ]:
property_value_df['units'] = property_value_df['units'].replace('', pd.NA).fillna(property_value_df['property'])

In [ ]:
property_value_df['units'] = property_value_df['units'].replace("Площа", "м2")

In [ ]:
property_value_df['units'] = property_value_df['units'].str.strip()

In [ ]:
property_value_df['units'].unique()

In [31]:
property_value_df.drop_duplicates()

,id,aimap_id,property,property_value,normalize_property,units
4,5,3,Кількість поверхів,26,Кількість поверхів,26
5,6,3,Площа,149296.6,Площа,149296.6
6,7,4,Кількість поверхів,1,Кількість поверхів,1
7,8,4,Площа,279.4,Площа,279.4
8,9,5,Кількість поверхів,1,Кількість поверхів,1
...,...,...,...,...,...,...
3353509,3650419,3052965,Фундамент,Бетон,Фундамент,Бетон
3353510,3650420,3052965,Підлоги,Ламінат,Підлоги,Ламінат
3353511,3650421,3052965,Покриття,Руберойд,Покриття,Руберойд
3353512,3650422,3052965,Сходи,Бетонні,Сходи,Бетонні
